In [1]:
import numpy as np
import pandas as pd
from qiskit.quantum_info import SparsePauliOp
import pennylane as qml

In [2]:
def create_matrix(cut_off, type):
    # Initialize a zero matrix of the specified size
    matrix = np.zeros((cut_off, cut_off), dtype=np.complex128)
    
    # Fill the off-diagonal values with square roots of integers
    for i in range(cut_off):
        if i > 0:  # Fill left off-diagonal
            if type == 'q':
                matrix[i][i - 1] = (1/np.sqrt(2)) * np.sqrt(i)  # sqrt(i) for left off-diagonal
            else:
                matrix[i][i - 1] = (1j/np.sqrt(2)) * np.sqrt(i)

        if i < cut_off - 1:  # Fill right off-diagonal
            if type == 'q':
                matrix[i][i + 1] = (1/np.sqrt(2)) * np.sqrt(i + 1)  # sqrt(i + 1) for right off-diagonal
            else:
                matrix[i][i + 1] = (-1j/np.sqrt(2)) * np.sqrt(i + 1)

    return matrix


In [3]:
# Function to calculate the Hamiltonian
def calculate_Hamiltonian(cut_off, potential):
    # Generate the position (q) and momentum (p) matrices
    q = create_matrix(cut_off, 'q')  # q matrix
    p = create_matrix(cut_off, 'p')  # p matrix

    # Calculate q^2 and q^3 for potential terms
    q2 = np.matmul(q, q)
    q3 = np.matmul(q2, q)

    #fermionic identity
    I_f = np.eye(2)

    #bosonic identity
    I_b = np.eye(cut_off)

    # Superpotential derivatives
    if potential == 'QHO':
        W_prime = q  # W'(q) = q
        W_double_prime = I_b #W''(q) = 1

    elif potential == 'AHO':
        W_prime = q + q3  # W'(q) = q + q^3
        W_double_prime = I_b + 3 * q2  # W''(q) = 1 + 3q^2

    elif potential == 'DW':
        W_prime = q + q2 + I_b  # W'(q) = q + q^2 + 1
        W_double_prime = I_b + 2 * q  # W''(q) = 1 + 2q

    else:
        print("Not a valid potential")
        raise

    # Kinetic term: p^2
    p2 = np.matmul(p, p)

    # Commutator term [b^†, b] = -Z
    Z = np.array([[1, 0], [0, -1]])  # Pauli Z matrix for fermion number
    commutator_term = np.kron(Z, W_double_prime)

    # Construct the block-diagonal kinetic term (bosonic and fermionic parts)
    # Bosonic part is the same for both, hence we use kron with the identity matrix
    kinetic_term = np.kron(I_f, p2)

    # Potential term (W' contribution)
    potential_term = np.kron(I_f, np.matmul(W_prime, W_prime))

    # Construct the full Hamiltonian
    H_SQM = 0.5 * (kinetic_term + potential_term + commutator_term)
    H_SQM[np.abs(H_SQM) < 10e-12] = 0
    
    return H_SQM

In [4]:
data = {'cut_off':[],
       'QHO': [],
       'AHO': [],
       'DW': []
       }

num_cut_offs = 10
cut_off_list = [2**i for i in range(1, num_cut_offs + 1)]
potential_list = ['QHO', 'AHO', 'DW']

for cut_off in cut_off_list:
    data['cut_off'].append(cut_off)

    for potential in potential_list:

        H = calculate_Hamiltonian(cut_off, potential)
        H_pauli = SparsePauliOp.from_operator(H)

        data[potential].append(H_pauli.size)

In [5]:
data

{'cut_off': [2, 4, 8, 16, 32, 64, 128, 256, 512, 1024],
 'QHO': [2, 4, 8, 16, 32, 64, 128, 256, 512, 1024],
 'AHO': [2, 10, 34, 102, 270, 670, 1548, 3496, 7486, 15534],
 'DW': [4, 14, 48, 136, 352, 854, 1990, 4450, 9874, 21202]}

In [6]:
pauli_df = pd.DataFrame(data).set_index('cut_off')

In [7]:
pauli_df

,QHO,AHO,DW
cut_off,,,
2,2,2,4
4,4,10,14
8,8,34,48
16,16,102,136
32,32,270,352
64,64,670,854
128,128,1548,1990
256,256,3496,4450
512,512,7486,9874


In [28]:
H = calculate_Hamiltonian(16, 'AHO')
H_pauli = SparsePauliOp.from_operator(H)
H_pauli.simplify(atol=1e-02).size

96

In [33]:
qml.pauli_decompose(H)

(
    1075.3124999999995 * (I(0) @ I(1) @ I(2) @ I(3) @ I(4))
  + 828.8468920554258 * (I(0) @ I(1) @ I(2) @ X(3) @ I(4))
  + 18.43918637876135 * (I(0) @ I(1) @ I(2) @ X(3) @ Z(4))
  + -19.06249999999997 * (I(0) @ I(1) @ I(2) @ Z(3) @ I(4))
  + -115.06249999999997 * (I(0) @ I(1) @ I(2) @ Z(3) @ Z(4))
  + 340.83168739458273 * (I(0) @ I(1) @ X(2) @ I(3) @ I(4))
  + -24.758217586165927 * (I(0) @ I(1) @ X(2) @ I(3) @ Z(4))
  + 492.29311898327876 * (I(0) @ I(1) @ X(2) @ X(3) @ I(4))
  + -64.14382455023042 * (I(0) @ I(1) @ X(2) @ X(3) @ Z(4))
  + -71.57612625320508 * (I(0) @ I(1) @ X(2) @ Z(3) @ I(4))
  + -13.379429045558396 * (I(0) @ I(1) @ X(2) @ Z(3) @ Z(4))
  + 438.1279732887981 * (I(0) @ I(1) @ Y(2) @ Y(3) @ I(4))
  + -56.76809525271241 * (I(0) @ I(1) @ Y(2) @ Y(3) @ Z(4))
  + -321.56249999999983 * (I(0) @ I(1) @ Z(2) @ I(3) @ I(4))
  + -109.81249999999993 * (I(0) @ I(1) @ Z(2) @ I(3) @ Z(4))
  + -296.51957069773084 * (I(0) @ I(1) @ Z(2) @ X(3) @ I(4))
  + -97.98193376051758 * (I(0) @ I(

In [19]:
H_pauli

SparsePauliOp(['IIII', 'IIXI', 'IIXZ', 'IIZI', 'IIZZ', 'IXII', 'IXIZ', 'IXXI', 'IXXZ', 'IXZI', 'IXZZ', 'IYYI', 'IYYZ', 'IZII', 'IZIZ', 'IZXI', 'IZXZ', 'IZZI', 'IZZZ', 'ZIII', 'ZIXI', 'ZIXZ', 'ZIZI', 'ZIZZ', 'ZXXI', 'ZXXZ', 'ZYYI', 'ZYYZ', 'ZZII', 'ZZIZ', 'ZZXI', 'ZZXZ', 'ZZZI', 'ZZZZ'],
              coeffs=[ 1.13312500e+02+0.j,  8.81890635e+01+0.j, -7.15058155e-02+0.j,
 -9.06250000e+00+0.j, -2.50625000e+01+0.j,  2.80566466e+01+0.j,
 -6.76713867e+00+0.j,  4.21567483e+01+0.j, -1.41466758e+01+0.j,
 -1.65897126e+01+0.j,  1.11774278e+00+0.j,  3.90996929e+01-0.j,
 -1.27666714e+01+0.j, -8.05625000e+01+0.j, -1.38125000e+01+0.j,
 -7.40404151e+01+0.j, -7.97834659e+00+0.j, -1.66875000e+01+0.j,
  3.40625000e+01+0.j,  5.75000000e+00+0.j,  2.96656305e+00+0.j,
 -3.82273369e-01+0.j, -7.50000000e-01+0.j, -7.50000000e-01+0.j,
  1.48804454e+00+0.j, -1.89006439e-01+0.j,  1.48804454e+00-0.j,
 -1.89006439e-01+0.j, -2.25000000e+00+0.j, -7.50000000e-01+0.j,
 -1.51767431e+00+0.j, -5.95519820e-03+0.j, -7.50000

In [26]:
H_pauli.size

102

In [10]:
from qiskit.quantum_info import Z2Symmetries

In [11]:
symmetries = Z2Symmetries.find_z2_symmetries(H_pauli)
tapered_hamiltonian = symmetries.taper(H_pauli)

In [12]:
H = sum(tapered_hamiltonian)

In [13]:
H.size

10

In [14]:
H

SparsePauliOp(['I', 'X', 'Z', 'I', 'X', 'Z', 'I', 'X', 'I', 'X'],
              coeffs=[13.0625    +0.j,  6.09879599+0.j, -9.625     +0.j,  7.5625    +0.j,
  3.97747564+0.j, -6.625     +0.j, 13.0625    +0.j, 10.56342452+0.j,
  7.5625    +0.j,  6.8891899 +0.j])

In [79]:
np.sort(np.linalg.eig(H)[0])

array([ 9.28279701+0.j, 73.21720299+0.j])

In [33]:
len(H2)

32

In [25]:
len(H2)

1

In [ ]:
data = {'cut_off':[],
       'QHO': [],
       'AHO': [],
       'DW': []
       }

num_cut_offs = 10
cut_off_list = [2**i for i in range(1, num_cut_offs + 1)]
potential_list = ['QHO', 'AHO', 'DW']

for cut_off in cut_off_list:
    data['cut_off'].append(cut_off)

    for potential in potential_list:

        H = calculate_Hamiltonian(cut_off, potential)
        H_pauli = SparsePauliOp.from_operator(H)

        data[potential].append({'hamiltonian_size': H_pauli.dim,
                        'num_paulis': len(H_pauli),
                        'num_qubits': H_pauli.num_qubits})

